# **TFM - Alessandro Bigolin - Parte 1a: Construcción del conjunto de datos**

## **0. Introducción**

NB: En caso de problemas con la visualización del notebook, puedes encontrarlo completamente ejecutado aquí:
https://colab.research.google.com/drive/1nawLN8tpm6-lxwaZaK8qcN361mtuG3w3?usp=sharing


### **0.1 Objetivos principales y fuentes de los datos**

Este TFM tiene como objetivo diseñar y ejecutar un proyecto de datos integral sobre el tráfico ferroviario en la región del Véneto en Italia, cubriendo el período de enero a mayo de 2025 (inclusive).

Concretamente, haré lo siguiente:
1. Destacar patrones recurrentes de retrasos;
2. Construir un modelo predictivo de ocurrencia de retrasos;
3. Publicar una herramienta visual e interactiva en Tableau Public, que haga accesibles los hallazgos.

El proyecto fue concebido inicialmente para la red de Renfe Rodalies en el área metropolitana de Barcelona. Sin embargo, la ausencia de datos abiertos para esta red lo hizo inviable. Por el contrario, Trenitalia (el principal operador ferroviario de Italia) proporciona datos abiertos a través de [ViaggiaTreno](http://www.viaggiatreno.it/infomobilita/index.jsp). Un servicio independiente, [TrainStats](https://trainstats.altervista.org/), luego procesa y pone a disposición del público estos datos en archivos JSON diarios que contienen información detallada sobre todos los movimientos de trenes en Italia.

Elegí centrarme en la región del Véneto por dos razones principales. En primer lugar, es mi región de origen, lo que me permite interpretar los datos teniendo en cuenta su geografía e infraestructura local. En segundo lugar, el conjunto de datos de Véneto para el período seleccionado de cinco meses ya comprende aproximadamente 1,5 millones de registros. Ampliar el análisis a toda la red ferroviaria italiana habría generado un conjunto de datos de tamaño excesivo para los recursos disponibles en este proyecto.  

### **0.2 En este notebook**
En esta primera fase del trabajo , me centro en recopilar los datos:

1. Comienzo realizando *web scraping* en una página web de Trenitalia, para generar un DataFrame que liste todas las estaciones con servicio en el Véneto. Luego, enriquezco esta lista con coordenadas geográficas para cada estación mediante llamadas API a Google Maps y guardo los datos como un archivo CSV (conjunto de datos `stations_list`).
2. `stations_list` se utiliza luego como conjunto de datos auxiliar para filtrar los archivos JSON diarios obtenidos de TrainStats. Los datos de los archivos JSON se filtran para retener solo los viajes de tren que incluyeron al menos una parada en una estación véneta. Finalmente, los datos filtrados de todos los archivos diarios se combinan en un único conjunto de datos coherente, guardado tanto como archivo JSON y como archivo CSV (conjunto de datos principal).

### **0.3 Importaciones y acceso a Google Drive**

In [ ]:
!pip install googlemaps

  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40714 sha256=3953e9ce2853629cff4e756490af1e9bce4a9b90530613ac01652a3f785fe530
  Stored in directory: /root/.cache/pip/wheels/4c/6a/a7/bbc6f5c200032025ee655deb5e163ce8594fa05e67d973aad6
Successfully built googlemaps


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import os
import json
import csv
from datetime import datetime
import googlemaps
from time import sleep

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **1. Creación del conjunto de datos auxiliar de la lista de estaciones**

En esta etapa creo un conjunto de datos que contiene todas las estaciones con servicio de Trenitalia en la región del Véneto. El proceso comienza con el *web scraping* de la página web oficial de Trenitalia que lista estas estaciones, extrayendo sus nombres, direcciones, localidades y provincias. Los nombres de las estaciones en bruto se normalizan para garantizar una codificación de caracteres coherente (por ejemplo, reemplazando caracteres acentuados). Posteriormente, el conjunto de datos se enriquece con coordenadas geográficas (latitud y longitud) obtenidas a través de llamadas a la API de Google Maps, lo que permitirá realizar análisis espaciales y geovisualización en etapas posteriores del proyecto. La lista final enriquecida se almacena como *stations_list_1a.csv*.

In [ ]:
url = "https://www.trenitalia.com/it/regionale/veneto/stazioni-servite-da-trenitalia-veneto.html"
req = requests.get(url)

soup = BeautifulSoup(req.text, "html.parser")

In [ ]:
# Finding div with id 'tableTrain-container'
table_div = soup.find(id="tableTrain-container")

if table_div:
    # Extracting the value of the data-table attribute
    data_table_str = table_div.get("data-table")
    if data_table_str:
        rows_html_list = json.loads(data_table_str)["data"]

        def replace_accents_with_apostrophe(text):
            replacements = {
                "à": "a'", "è": "e'", "é": "e'", "ì": "i'", "ò": "o'", "ù": "u'",
                "À": "A'", "È": "E'", "É": "E'", "Ì": "I'", "Ò": "O'", "Ù": "U'"
            }
            for accented_char, replacement in replacements.items():
                text = text.replace(accented_char, replacement)
            return text

        stations = []

        for row_html in rows_html_list:
            row_soup = BeautifulSoup(row_html, "html.parser")
            tds = row_soup.find_all("td")

            if len(tds) >= 4:
                nome = tds[0].get_text(strip=True)
                indirizzo = tds[1].get_text(strip=True)
                comune = tds[2].get_text(strip=True)
                provincia = tds[3].get_text(strip=True)

                nome = replace_accents_with_apostrophe(nome)

                stations.append({
                    "station": nome,
                    "address": indirizzo,
                    "town": comune,
                    "province": provincia
                })

# Putting the data in a dataframe
df_stations = pd.DataFrame(stations, columns=["station", "address", "town", "province"])

df_stations.head()

,station,address,town,province
0,Abano,"Via della Stazione, 10",Abano Terme,PD
1,Adria,"Via Umberto Maddalena, 32",Adria,RO
2,Alano-Fener-Valdobbiadene,"Via Stazione, 106",Alano di Piave,BL
3,Albaredo,"Via Stazione, 60",Vedelago,TV
4,Altavilla-Tavernelle,"Piazzale Stazione, 35",Altavilla Vicentina,VI


In [ ]:
# Adding coordinates for each station through API calls to Google Maps
API_KEY = "***"
gmaps = googlemaps.Client(key=API_KEY)

latitudes = []
longitudes = []

for index, row in df_stations.iterrows():
    query = f"{row['station']}, {row['town']}, {row['province']}, Italia"
    try:
        geocode_result = gmaps.geocode(query)
        if geocode_result:
            location = geocode_result[0]['geometry']['location']
            latitudes.append(location['lat'])
            longitudes.append(location['lng'])
        else:
            latitudes.append(None)
            longitudes.append(None)
    except Exception as e:
        print(f"Errore per {query}: {e}")
        latitudes.append(None)
        longitudes.append(None)
    sleep(0.1)

df_stations["lat"] = latitudes
df_stations["lon"] = longitudes

# Saving the dataframe as a csv file
df_stations.to_csv("/content/drive/MyDrive/Trains_project/stations_list_1a.csv", sep=";", index=False)

In [ ]:
df_stations.head()

,station,address,town,province,lat,lon
0,Abano,"Via della Stazione, 10",Abano Terme,PD,45.362136,11.790235
1,Adria,"Via Umberto Maddalena, 32",Adria,RO,45.055549,12.056038
2,Alano-Fener-Valdobbiadene,"Via Stazione, 106",Alano di Piave,BL,45.902645,11.944375
3,Albaredo,"Via Stazione, 60",Vedelago,TV,45.666169,12.012208
4,Altavilla-Tavernelle,"Piazzale Stazione, 35",Altavilla Vicentina,VI,45.511801,11.454374


## **2. Creación del conjunto de datos principal**

En esta sección se construye el conjunto de datos principal de paradas de tren mediante el filtrado de los archivos JSON diarios de movimientos ferroviarios a nivel nacional en Italia (enero-mayo de 2025), disponibles públicamente. El filtrado conserva únicamente aquellos trenes que efectuaron al menos una parada en una estación del Véneto. Para ello, se emplea como referencia el conjunto de datos `stations_list` elaborado previamente,

Todos los registros diarios filtrados se agregan primero en un único archivo JSON, conservando su estructura día a día. Luego, los datos se transforman en un formato CSV tabular, extrayendo para cada parada atributos relevantes como el número de tren, la categoría, el origen, el destino, el nombre de la estación, los horarios de llegada y salida programados, y los retrasos en la llegada y la salida.

Los archivos JSON diarios puestos a disposición por TrainStats se componen de las siguientes secciones principales:

- **giorno**: Fecha de referencia (por ejemplo, `"31/05/2025"`)
- **timeZone**: Desplazamiento de la zona horaria en segundos (por ejemplo, `7200 = UTC+2`)
- **riassunto**: Estadísticas agregadas del día
- **avvisiRFI**: Avisos oficiales de RFI (administrador de infraestructura)
- **avvisiTI**: Avisos oficiales de Trenitalia (operador ferroviario)
- **treni**: Lista detallada de trenes monitorizados

Para la construcción del conjunto de datos, me centraré en la última sección.

**Sección `"treni"`:**

Array de objetos, cada uno representando un tren monitorizado:

- **_id**: Identificador único
- **n**: Número de tren
- **p**: Estación de salida
- **rp**: Retraso en la salida
- **a**: Estación de llegada
- **ra**: Retraso en la llegada
- **c**: Categoría (*REG = Regional, IC = InterCity, EC = EuroCity, EN = EuroNight, ICN = InterCity Notte, etc.*)
- **op**: Hora de salida programada (unix timestamp)
- **oa**: Hora de llegada programada (unix timestamp)
- **fr**: Array de paradas

Cada parada (**fr**) contiene:
- **n**: Nombre de la estación
- **ra**: Retraso en la llegada
- **rp**: Retraso en la salida
- **br**: Número de andén real
- **oa**: Hora de llegada programada (unix timestamp)
- **op**: Hora de salida programada (unix timestamp)

Iteraré a través de todos los archivos JSON diarios que cubren el período del 1 de enero al 31 de mayo, filtraré los viajes manteniendo solo aquellos que incluyen al menos una estación en `stations_list` (lista de estaciones venetas), y luego agregaré todos los viajes en un único archivo, guardado tanto como *trains_data_Veneto.json* como *trains_data_Veneto_1a.csv*.

In [ ]:
# Accessing the folder with the daily JSON files
json_dir = '/content/drive/MyDrive/Trains_project/original_JSON/'

# Setting the valid stations from the df_stations dataset
valid_stations = set(df_stations['station'].dropna().str.upper().unique())

# list of days with filtered trains
aggregated_data = []

# Looping through all .json files in the folder
for filename in os.listdir(json_dir):
    if filename.endswith('.json'):
        json_path = os.path.join(json_dir, filename)

        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        filtered_day = {
            "day": data.get("giorno", "n.d."),
            "trains": []
        }

        for train in data.get("treni", []):
            stops = train.get("fr", [])
            train_stops = {f["n"].upper() for f in stops if "n" in f}

            if valid_stations.intersection(train_stops):
                filtered_day["trains"].append(train)

        if filtered_day["trains"]:
            aggregated_data.append(filtered_day)

# Saving a single JSON file with all the filtered data from the daily JSON files, day by day
output_path = '/content/drive/MyDrive/Trains_project/trains_data_Veneto.json'
with open(output_path, 'w', encoding='utf-8') as f_out:
    json.dump(aggregated_data, f_out, ensure_ascii=False, indent=2)

**De JSON a CSV:**

In [ ]:
# Loading JSON
with open(output_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# CSV columns names
fieldnames = [
    "day", "train_number", "category", "origin", "destination",
    "station", "scheduled_arrival", "scheduled_departure",
    "arrival_delay", "departure_delay"
]

csv_path = '/content/drive/MyDrive/Trains_project/trains_data_Veneto_1a.csv'

with open(csv_path, "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=';')
    writer.writeheader()

    for day_data in data:
        day = day_data.get("day", "")
        for train in day_data.get("trains", []):
            train_number = train.get("n", "")
            category = train.get("c", "")
            origin = train.get("p", "")
            destination = train.get("a", "")
            stops = train.get("fr", [])

            for stop in stops:
                oa = stop.get("oa")
                op = stop.get("op")

                writer.writerow({
                    "day": day,
                    "train_number": train_number,
                    "category": category,
                    "origin": origin,
                    "destination": destination,
                    "station": stop.get("n", ""),
                    "scheduled_arrival": oa,
                    "scheduled_departure": op,
                    "arrival_delay": stop.get("ra", ""),
                    "departure_delay": stop.get("rp", "")
                })

In [ ]:
trains_df = pd.read_csv(csv_path, sep=';')

In [ ]:
trains_df.tail()

,day,train_number,category,origin,destination,station,scheduled_arrival,scheduled_departure,arrival_delay,departure_delay
1478780,31/05/2025,2647,REG,MILANO CENTRALE,VERONA PORTA NUOVA,ROVATO,1748729940,1748730000,-2,1
1478781,31/05/2025,2647,REG,MILANO CENTRALE,VERONA PORTA NUOVA,BRESCIA,1748730660,1748730780,-2,1
1478782,31/05/2025,2647,REG,MILANO CENTRALE,VERONA PORTA NUOVA,DESENZANO DEL GARDA-SIRMIONE,1748731680,1748731740,1,2
1478783,31/05/2025,2647,REG,MILANO CENTRALE,VERONA PORTA NUOVA,PESCHIERA DEL GARDA,1748732220,1748732280,1,3
1478784,31/05/2025,2647,REG,MILANO CENTRALE,VERONA PORTA NUOVA,VERONA PORTA NUOVA,1748733420,0,-6,N
